<a href="https://colab.research.google.com/github/strawndri/python-ds-pandas-limpeza-tratamento-dados/blob/main/Desafios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Conhecendo os dados

Você recebeu a tarefa de normalizar o seguinte arquivo JSON chamado `informacoes.json` referente a algumas informações de identificação de um cliente. O arquivo possui o seguinte conteúdo:

```
{
  "nome": "João",
  "idade": 28,
  "enderecos": [
    {
      "tipo": "casa",
      "rua": "Rua A",
      "numero": 123,
      "cidade": "São Paulo"
    },
    {
      "tipo": "trabalho",
      "rua": "Rua B",
      "numero": 456,
      "cidade": "Rio de Janeiro"
    }
  ]
}
```

O resulto final deve ser um Pandas DataFrame.

In [1]:
import pandas as pd
import json

In [2]:
with open('informacoes.json') as f:
  json_bruto = json.load(f)

In [3]:
json_bruto

{'nome': 'João',
 'idade': 28,
 'enderecos': [{'tipo': 'casa',
   'rua': 'Rua A',
   'numero': 123,
   'cidade': 'São Paulo'},
  {'tipo': 'trabalho',
   'rua': 'Rua B',
   'numero': 456,
   'cidade': 'Rio de Janeiro'}]}

O parâmetro `record_path` é usado para especificar o caminho para os dados que queremos normalizar. No nosso caso, queremos normalizar os dados da lista de endereços, então especificamos `'enderecos'`.

Além disso, o parâmetro `meta` é usado para especificar quais chaves do objeto original queremos incluir na saída normalizada. Optamos por incluir `'nome'` e `'idade'`.

In [6]:
dados_normalizados = pd.json_normalize(json_bruto, record_path='enderecos', meta=['nome', 'idade'])
dados_normalizados.head()

,tipo,rua,numero,cidade,nome,idade
0,casa,Rua A,123,São Paulo,João,28
1,trabalho,Rua B,456,Rio de Janeiro,João,28
